In [3]:
from konlpy.tag import Mecab
mecab = Mecab()
print(mecab.morphs(u'영등포구청역에 있는 맛집 좀 알려주세요.'))
print(mecab.nouns(u'영등포구청역에 있는 맛집 좀 알려주세요.'))
print(mecab.pos(u'영등포구청역에 있는 맛집 좀 알려주세요.'))

['영등포구청역', '에', '있', '는', '맛집', '좀', '알려', '주', '세요', '.']
['영등포구청역', '맛집']
[('영등포구청역', 'NNP'), ('에', 'JKB'), ('있', 'VV'), ('는', 'ETM'), ('맛집', 'NNG'), ('좀', 'MAG'), ('알려', 'VV+EC'), ('주', 'VX'), ('세요', 'EP+EF'), ('.', 'SF')]


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns

In [30]:
train_df = pd.read_csv('https://raw.githubusercontent.com/haram4th/ablearn/main/ratings_train.txt', sep='\t')

In [7]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [9]:
test_df = pd.read_csv('./data/ratings_test.csv',sep='\t')
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [10]:
train_df['label'].value_counts()

label
0    75173
1    74827
Name: count, dtype: int64

In [11]:
test_df['label'].value_counts()

label
1    25173
0    24827
Name: count, dtype: int64

In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [32]:
train_df = train_df.dropna()
test_df = test_df.dropna()

In [17]:
train_df['document']

0                                       아 더빙.. 진짜 짜증나네요 목소리
1                         흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                             교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
                                ...                        
149995                                  인간이 문제지.. 소는 뭔죄인가..
149996                                        평점이 너무 낮아서...
149997                      이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?
149998                          청춘 영화의 최고봉.방황과 우울했던 날들의 자화상
149999                             한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 149995, dtype: object

## 정규표현식을 이용한 텍스트 정리

In [18]:
import re

In [26]:
def text_clean(x):
    pattern = r'[가-힣0-9a-zA-Z]+'
    matches = re.findall(pattern, x)
    matches = ' '.join(matches)
    return matches

In [33]:
train_df['document'] = train_df['document'].apply(text_clean)

In [29]:
test_df['document'] = test_df['document'].apply(text_clean)

In [58]:
X = train_df['document']
y = train_df['label']

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

## CountVectorizer()
* document에 있는 문자를 숫자로 변환 / 벡터화

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

In [39]:
cv = CountVectorizer() # 토크나이저가 공백 기주으로 토큰화
X_train = cv.fit_transform(X_train)
X_valid = cv.transform(X_valid)

### 독립변수가 문자로 되어 있는 데이터의 경우 MultinomialNB

In [42]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [44]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
pred = mnb.predict(X_valid)
print(classification_report(pred, y_valid))

              precision    recall  f1-score   support

           0       0.84      0.81      0.82     23267
           1       0.81      0.83      0.82     21732

    accuracy                           0.82     44999
   macro avg       0.82      0.82      0.82     44999
weighted avg       0.82      0.82      0.82     44999



## test 데이터 불러와서 검증

In [46]:
X_test = test_df['document']
y_test = test_df['label']

In [48]:
X_test_vec = cv.transform(X_test)

In [49]:
test_pred = mnb.predict(X_test_vec)
print(classification_report(test_pred, y_test))

              precision    recall  f1-score   support

           0       0.84      0.81      0.82     25707
           1       0.81      0.84      0.82     24290

    accuracy                           0.82     49997
   macro avg       0.82      0.82      0.82     49997
weighted avg       0.82      0.82      0.82     49997



# KoNLPy의 토크나이저 Mecab을 이용해 형태소 분리

In [50]:
from konlpy.tag import Mecab
mecab = Mecab()

In [51]:
def tokenizer(text):
    tokens = mecab.morphs(text)
    return tokens

## CountVectorizer
* 모든 단어를 벡터화시켜줌
## TfidVectorizer
* 자주 등장하는 단어에 가중치

### 1) CountVectorizer: 단어 빈도(횟수) 기반 벡터화
* 작동 방식: CountVectorizer는 각 문서에서 등장한 단어의 **횟수(빈도)**를 기준으로 벡터를 만듭니다. 단순히 각 문서에서 특정 단어가 몇 번 등장했는지를 세어 벡터화합니다.<br>
* 특징:
    1) 각 단어의 빈도가 높을수록 해당 단어의 중요성이 더 크다고 가정합니다.<br>
    2) 빈도가 높다는 것만을 고려하기 때문에, 모든 문서에서 자주 등장하는 단어도 중요한 단어로 처리될 수 있습니다.<br>
* 예시:<br>
* 예를 들어, 두 개의 문서가 있다면:<br>
* 문서 1: "고양이가 나무 위에 있다."<br>
* 문서 2: "나무 아래에 고양이가 있다."<br>
* 이 두 문서를 CountVectorizer로 변환하면 단어 빈도가 포함된 벡터가 생성됩니다:<br>
    * ['고양이': 2, '나무': 2, '위에': 1, '아래에': 1, '있다': 2]<br>
### 2) TfidfVectorizer: TF-IDF (Term Frequency-Inverse Document Frequency) 기반 벡터화
* 작동 방식: TfidfVectorizer는 단어의 빈도뿐만 아니라, 단어의 중요도를 계산합니다. 여기서는 TF-IDF 값을 사용하여 문서 간 차별성을 강조합니다.<br>
* TF (Term Frequency): 단어가 문서에서 얼마나 자주 등장했는지를 나타냅니다.<br>
* IDF (Inverse Document Frequency): 단어가 다른 문서에 얼마나 자주 등장하지 않았는지를 나타냅니다. 자주 등장하지 않는 단어는 더 중요한 단어로 간주합니다.<br>
* 특징:<br>
    1) TF-IDF는 문서 전체에서 자주 등장하는 흔한 단어들(예: "그리고", "이다" 등)의 중요도를 낮추고, 문서에서만 중요한 단어들의 중요도를 높입니다.<br>
    2) 단순히 빈도가 높은 단어보다 특정 문서에서 더 특징적인 단어에 더 높은 가중치를 부여합니다.<br>
* 예시:<br>
* 위의 문서 1과 문서 2에 대해 TfidfVectorizer로 변환하면, 공통 단어들(예: "있다", "고양이")의 중요도는 낮아지고, 차별적인 단어(예: "위에", "아래에")의 중요도는 상대적으로 높아집니다.

In [52]:
# countervectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [53]:
doc = ['고양이가 나무 위에 있다.', '나무 아래에 고양이가 있다.']

In [55]:
c_vec = CountVectorizer()
X = c_vec.fit_transform(doc)
print(c_vec.get_feature_names_out()) # 벡터화된 단어 목록
print(X.toarray()) # 단어 빈도 벡터

['고양이가' '나무' '아래에' '위에' '있다']
[[1 1 0 1 1]
 [1 1 1 0 1]]


In [56]:
# tfidvectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [57]:
tfidf = TfidfVectorizer()
X2 = tfidf.fit_transform(doc)
print(tfidf.get_feature_names_out()) # 벡터화된 단어 목록
print(X2.toarray()) # 단어 빈도 벡터

['고양이가' '나무' '아래에' '위에' '있다']
[[0.44832087 0.44832087 0.         0.63009934 0.44832087]
 [0.44832087 0.44832087 0.63009934 0.         0.44832087]]


### CountVectorizer와 TfidfVectorizer 비교

| **특성**             | **CountVectorizer**                                     | **TfidfVectorizer**                                               |
|----------------------|--------------------------------------------------------|-------------------------------------------------------------------|
| **기반**             | 단어의 단순 빈도                                        | 단어 빈도 + 문서 내에서의 상대적 중요도(TF-IDF)                     |
| **단어 빈도 계산**    | 문서에서 등장한 단어의 단순한 등장 횟수를 셈            | 단어의 등장 횟수(TF)와 해당 단어가 문서들에서 얼마나 자주 등장하지 않았는지를 함께 고려(IDF) |
| **빈번한 단어 처리**  | 문서에서 자주 등장하는 단어일수록 높은 가중치를 부여    | 문서에서 흔한 단어는 가중치를 낮추고, 드문 단어는 높은 가중치를 부여   |
| **주요 용도**        | 단순한 단어 빈도 기반 분석이 필요할 때 사용             | 문서 간 차별적인 단어를 구별할 때 유용                                |
| **계산 비용**         | 상대적으로 적음                                         | 상대적으로 더 복잡하고 계산 비용이 높음                                |


* tokenizer : 형태소를 분석해서 단어를 나누어 주는 역할, 기본은 공백 기준으로 나눔
    * 한국어는 조사가 있기 때문에 공백이 아닌 형태소 분석을 통해서 나눠야함
    * konlpy의 형태소 분석기를 이용해서 형태소를 나누고 분석
* ngram_range(1,2)
    * 단어를 벡터화할 때 단어의 범위를 지정
    * (1,2) → 1-gram, 2-gram
    * "이 영화는 정말 좋다"
    * 1-gram : ['이','영화는','정말','좋다']
    * 2-gram : ['이 영화는','영화는 정말','정말 좋다']
* min_df=3:
    * 단어가 등장하는 최소 문서 수를 설정하는 파리미터
* max_df=0.9:
    * 단어가 전체 문서의 90% 이하에서 등장할 때만 벡터화에 포함

In [59]:
tf_X_train, tf_X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

In [60]:
#tokenizer를 konlpy의 mecab사용
tfidf = TfidfVectorizer(tokenizer=text_clean, ngram_range=(1,2), min_df=4, max_df=0.9)
tf_X_train = tfidf.fit_transform(tf_X_train)
tf_X_valid = tfidf.transform(tf_X_valid)

/home/ceun2/miniforge3/envs/fintech/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [61]:
mnb = MultinomialNB()
mnb.fit(tf_X_train, y_train)
pred = mnb.predict(tf_X_valid)
print(classification_report(pred, y_valid))

              precision    recall  f1-score   support

           0       0.85      0.84      0.85     23012
           1       0.84      0.85      0.84     21987

    accuracy                           0.84     44999
   macro avg       0.84      0.84      0.84     44999
weighted avg       0.84      0.84      0.84     44999



In [62]:
tf_X_test =tfidf.transform(X_test)
test_pred = mnb.predict(tf_X_test)
print(classification_report(test_pred, y_test))

              precision    recall  f1-score   support

           0       0.85      0.83      0.84     25590
           1       0.83      0.85      0.84     24407

    accuracy                           0.84     49997
   macro avg       0.84      0.84      0.84     49997
weighted avg       0.84      0.84      0.84     49997



## random_forest와 비교

In [63]:
from sklearn.ensemble import RandomForestClassifier

In [65]:
rfc = RandomForestClassifier(random_state=42, n_jobs=2)
rfc.fit(tf_X_train, y_train)
pred = rfc.predict(tf_X_valid)
print(classification_report(pred, y_valid))

              precision    recall  f1-score   support

           0       0.85      0.81      0.83     23561
           1       0.80      0.84      0.82     21438

    accuracy                           0.83     44999
   macro avg       0.83      0.83      0.83     44999
weighted avg       0.83      0.83      0.83     44999



In [66]:
test_pred = mnb.predict(tf_X_test)
print(classification_report(test_pred, y_test))

              precision    recall  f1-score   support

           0       0.85      0.83      0.84     25590
           1       0.83      0.85      0.84     24407

    accuracy                           0.84     49997
   macro avg       0.84      0.84      0.84     49997
weighted avg       0.84      0.84      0.84     49997

